##  Study on how to give initial data. 

We want to have a very isotropic data with a given frequency, or perhaps a distribution (thermal?) for $phi$ and its time derivative. Here
We do in 2dim.

We first create a distribution of values on Fourier space both for $ϕ $ and its time derivative, $\phi_t$.
I think we need a thermal distribution, that is we fix a temperature and put particles at that temperature.
That is we shall use the $\omega = |k|$ rule, we take positive so as to have particles and not antiparticles.



In [29]:
using Plots
using Distributions
using FFTW
using LinearAlgebra
using Revise
using Base.Threads
using HDF5

In [2]:
includet("../PIC/PIC-1D/aux_functions/inidat_aux_functions.jl")
includet("../PIC/PIC-1D/aux_functions/aux_functions.jl")

In [ ]:
function get_index_p(x,J,L)
    floor(Int64,(x>=0) ? mod(x / L[1] * J[1] + 1 ,J[1]) : -mod(-x / L[1] * J[1] ,J[1])+J+1 )
    end
    
function get_index_p(x::Vector,J,L)
        j = Vector{Int64}(undef,length(J))
        for i in 1:length(J)
            j[i] = get_index_p(x[i],J[i],L[i])
        end
        return j[:]
end

function n(ϕ,ϕ_t,Box_x,J)
        V = volume(Box_x)
        dx = differentials(Box_x,J)
        n = real(im*sum(ϕ.*conj.(ϕ_t).-conj.(ϕ).*ϕ_t))/2#*prod(dx)/2)
        return n, n/prod(J)
end

We first look at the scheme for a distribution which resembles a delta function $k$ in Fourier space.
It gives a rather homogeneous distribution is space.

In [ ]:
#J = (501,501,501)
J = (240,240,240)
Box_x = [-120.0, 120.0, -120.0, 120.0, -120.0, 120.0]
dx = differentials(Box_x,J)
κ = 2π./(Box_x[2:2:end] - Box_x[1:2:end-1])
V = volume(Box_x)
ϕ = Array{ComplexF64}(undef,J...)
ϕ_t = copy(ϕ)
g = copy(ϕ)
g_t = copy(ϕ)
g_p = copy(ϕ)

m = 1
m2 = m^2

N = 1000 #number of particles
 
function f(k,f_pars)
    (k0,δ) = f_pars
    if abs(k'*k - k0^2) < δ^2
        return (1.0 + 0.0im)/(2π*(k0*2*δ)) # normalized so that the integral is 1
    else
        return 0.0 + 0.0im
    end
end

function h(k,h_pars)
    k0 = h_pars
    if abs(k'*k)< k0^2
        return (1.0 + 0.0im)/2*π/k0^2
    else
        return 0.0
    end
end

# k0= 500; δ = 1000 # para J=2001
k0 = 1; δ = 0.5

function get_frequencies(Box,J)
    κ = 2π./(Box_x[2:2:end] - Box_x[1:2:end-1])
    kx = fftfreq(J[1]).*κ[1]*J[1]
    ky = fftfreq(J[2]).*κ[2]*J[2]
    kz = fftfreq(J[3]).*κ[3]*J[3]
    return kx, ky, kz
end

kx, ky, kz = get_frequencies(Box_x,J)

kx_p = sort(kx)
ky_p = sort(ky)
kz_p = sort(kz)

ω(k,m2) = sqrt((k'*k+m2))

function get_fourier_data(f,Box_x,J,m2,f_pars)
    g = Array{ComplexF64}(undef,J...)
    g_p = copy(g)
    κ = 2π./(Box_x[2:2:end] - Box_x[1:2:end-1])

    kx, ky, kz = get_frequencies(Box_x,J)
    
    kx = fftfreq(J[1]).*κ[1]*J[1]
    ky = fftfreq(J[2]).*κ[2]*J[2]
    kz = fftfreq(J[3]).*κ[3]*J[3]
    
    kx_p = sort(kx)
    ky_p = sort(ky)
    kz_p = sort(kz)

    for i in 1:J[1]
        for j in 1:J[2]
            for l in 1:J[3]
            ω_val = ω([kx[i];ky[j];kz[l]],m2)
            g[i,j,l] = exp(im*2*π*rand())*f([kx[i],ky[j],kz[l]],f_pars...)/sqrt(2.0*ω_val)
            g_t[i,j,l] = im*sqrt(2.0*ω_val)*g[i,j,l]
            ω_val_p = ω([kx_p[i];ky_p[j];kz_p[l]],m2)
            g_p[i,j,l] = f([kx_p[i],ky_p[j],kz_p[l]],f_pars)*exp(im*2*π*rand())/sqrt(2.0*ω_val_p)
            end
        end
    end
    return g, g_p
end



In [ ]:
dx

In [ ]:
function policut(x,x0,r0,p)
    r2 = (x-x0)'*(x-x0)
    if (r0 - r2) >= 0.0
        return (r0 - r2)^p/r0^p
    else
        return 0.0
    end
end

function poli_step(x,x0,x1,p)
    if x < 0.0 || x0 < 0.0 || x1 < x0
        error("the variable is suposed to be positive")
    end
    if x0 < x && x < x0 + (x1-x0)/2.0
        return 1.0 - ((x-x0)^p/abs((x1-x0)/2.0)^p)/2.0 #+ (x1-x)^p/(x1-x0)^p
    elseif x >= x0 + (x1-x0)/2.0 && x < x1
        return +(x1-x)^p/abs((x1-x0)/2.0)^p/2.0
    elseif x <= x0
        return 1.0
    else
        return 0.0
    end
end

function step_cut(g,Box,J,p,percent,δ)
    x,y,z = get_coords(Box,J)
    g_cut = copy(g)
    x0 = (Box[2:2:end].+Box[1:2:end])./2.0
    L = (Box[2:2:end].-Box[1:2:end])
    r0 = minimum(abs.(L))*percent
    for i in 1:J[1]
        for j in 1:J[2]
            for k in 1:J[3]
                r = sqrt((x[i]-x0[1])^2+(y[j]-x0[2])^2+(z[k]-x0[3])^2)
                g_cut[i,j,k] = g[i,j,k] * poli_step(r,r0*(1.0-δ),r0,p)
            end
        end
    end
    return g_cut
end

function polinomial_cut(g,Box,J,p,percent)
    dx = differentials(Box,J)
    x,y,z = get_coords(Box,J)
    g_cut = copy(g)
    x0 = (Box[2:2:end].+Box[1:2:end])./2.0
    L = (Box[2:2:end].-Box[1:2:end])
    r0 = minimum(abs.(L))*percent
    r0 = r0^2
    for i in 1:J[1]
        for j in 1:J[2]
            for k in 1:J[3]
                g_cut[i,j,k] = g[i,j,k] * policut([x[i];y[j];z[k]],x0,r0,p)
            end
        end
    end
    return g_cut
end

function get_coords(Box,J)
    dx = differentials(Box,J)
    x = [Box_x[1] + (i-1)*dx[1] for i in 1:J[1]]
    y = [Box_x[3] + (i-1)*dx[2] for i in 1:J[2]]
    z = [Box_x[5] + (i-1)*dx[3] for i in 1:J[3]]
    return x, y ,z
end



In [ ]:
k0 = 1.2; δ = 0.5 #coarse k0=0.5
h_pars = k0
f_pars = (k0, δ)
#g, g_p = get_fourier_data(f,Box_x,J,1,f_pars)
g, g_p = get_fourier_data(h,Box_x,J,1,h_pars)

In [12]:
if false
    g = fill(0.0+im*0.0,J...)
    g_t = copy(g)

    g[5,5] = exp(im*2*π*rand())
    g_t[5,5] = im*(sqrt(kx[5]^2+ky[5]^2 + m2))*g[5,5]
end

In [ ]:

sum(abs.(g))*prod(κ)
heatmap(ky_p,kx_p,real.(g_p[:,:,J[3]÷2]), aspectratio = 1)
#surface(ky_p,kx_p,real.(g[:,:]), aspectratio = 1)


In [107]:
#ifft(g,J...)
p_i = plan_ifft(g; flags=FFTW.ESTIMATE, timelimit=Inf);

In [108]:
ϕ = p_i*g*sqrt(prod(J));
ϕ_t = p_i*g_t*sqrt(prod(J));


In [ ]:

sum(ϕ.*conj.(ϕ))/prod(J)

In [ ]:

@show norm(ϕ)
@show mean(ϕ)
@show var(ϕ);
@show n(ϕ,ϕ_t,Box_x,J);

In [ ]:
p = 4 #polinomial order
percent = 0.550 #0.6 goes up to the border in poli_cut use 0.550 for step_cut
δ = 0.3 #less than 1!
x,y,z = get_coords(Box_x,J)
#ϕ_h_cut = polinomial_cut(ϕ,Box_x,J,p,percent);
ϕ_h_step_cut = step_cut(ϕ,Box_x,J,p,percent,δ)
heatmap(x,y,real.(ϕ_h_step_cut[:,:,J[3]÷2]), aspectratio=1)
#heatmap(real.(ϕ_h_cut[J[1]÷2,:,:]), aspectratio=1)
#heatmap(real.(ϕ_h_cut[:,J[2]÷2,:]), aspectratio=1)
#heatmap(imag.(ϕ[:,J[2]÷2,:]), aspectratio=1)
#surface(x,y,real.(ϕ_h_step_cut[:,:,J[3]÷2]), aspectratio=1)
#png("h_cut.png")

In [119]:
#ϕ_h_cut = polinomial_cut(ϕ,Box_x,J,8);
#ϕ_t_h_cut = polinomial_cut(ϕ_t,Box_x,J,8);

ϕ_h_cut = step_cut(ϕ,Box_x,J,p,percent,δ)
ϕ_t_h_cut = step_cut(ϕ_t,Box_x,J,p,percent,δ)

N_fields = 4.0 #Int32(1)

x, y, z = get_coords(Box_x,J)

# Open an HDF5 file in write mode
h5file = h5open("example_data_3d_h_step_cut.h5", "w") do file
    write(file, "coord0", x)
    write(file, "coord1", y)
    write(file, "coord2", z)
    write(file, "nvars", [N_fields])
    write(file, "var0", real.(ϕ_h_cut))
    write(file, "var1", imag.(ϕ_h_cut))
    write(file, "var2", real.(ϕ_t_h_cut))
    write(file, "var3", imag.(ϕ_t_h_cut))
end

In [ ]:
data = h5open("example_data_3d_h_cut.h5", "r")

In [ ]:
var0 = read(data["var0"])
heatmap(x,y,var0[:,:,J[3]÷2], aspectratio=1)
#surface(x,y,real.(ϕ_h_cut)[:,:,J[3]÷2], aspectratio=1)

In [ ]:
J[3]÷2

In [ ]:
norm(var0)
var0[120,120,120]

## Thermal distribution

We use a relativistic thermal distribution. I guess we are setting the mass to 1 but I don't know the temperature.



In [ ]:
θ = 1.0
m = 1.0
m2 = m^2
norm_rel = 2.0
p_max = 10*θ
Box_p =(-p_max,p_max,-p_max,p_max,-p_max,p_max)
#dp = differentials(Box_p,J)
L = zeros(J...)
L = Box_p[2:2:end] .- Box_p[1:2:end]
f_p = f_p_rel 
f_p_max = f_p_rel_max
par_f_p_1 = (θ,norm_rel,2)
#norm_rel = int_mid_point_f(f_p, par_f_p_1, [20_000,20_000], Box_p)
par_f_p = (θ, norm_rel,2)
interval_p = Box_p


In [ ]:
g_th = fill(0.0+im*0.0,J...)
gt_th = fill(0.0+im*0.0,J...)

N=100000
plt = Plots.scatter(labels=false)
for i in 1:N÷2
    kr = random_sampling_from_distribution_D(f_p,f_p_max,par_f_p,interval_p)
    j = get_index_p(kr,J,L)
    jm = get_index_p(-kr,J,L)
    if (abs(j[1]) > J[1] || abs(jm[1])) > J[1] || (abs(j[2]) > J[2] || abs(jm[2])) > J[2]
        println("j = $j, jm = $jm, kr = $kr")
    end
    #Plots.scatter!(plt,[j[1],jm[1]],[j[2],jm[2]],label=false)
    factor = 1.0/sqrt(2*ω([kr[1],kr[2],kr[3]],m2))
    phase_p = exp(im*rand()*2*π)
    phase_m = exp(im*rand()*2*π)
    g_th[j...] += phase_p*factor
    g_th[jm...] += phase_m*factor
    gt_th[j...] += im*phase_p/factor
    gt_th[jm...] += im*phase_m/factor
end

#heatmap(real.(g_th))
surface(real.(g_th[30,:,:]))
#plt

In [ ]:
#heatmap(real.(g_th))
surface(abs.(imag.(g_th)).- abs.(real.(g_th)))

In [19]:
ϕ_th = p_i*g_th*sqrt(prod(J));
ϕt_th = p_i*gt_th*sqrt(prod(J));

In [ ]:
#heatmap(imag.(ϕ_th))
surface(imag.(ϕ_th[30,:,:]))

In [ ]:
#@show sum(ϕ.*conj.(ϕ))*prod(dx)/V

n(ϕ_th,ϕt_th,Box_x,J)

### Localization



In [71]:
ϕ_th_cut = polinomial_cut(ϕ_th,Box_x,J,8);

In [ ]:
surface(real.(ϕ_th_cut[101,:,:]) )

### Data transfer

In [24]:
using HDF5

In [25]:
N_fields = 4.0 #Int32(1)

x = [Box_x[1] + (i-1)*dx[1] for i in 1:J[1]]
y = [Box_x[3] + (i-1)*dx[2] for i in 1:J[2]]
z = [Box_x[5] + (i-1)*dx[3] for i in 1:J[3]];

# Open an HDF5 file in write mode
h5file = h5open("example_data_3d_gaussian.h5", "w") do file
    write(file, "coord0", x)
    write(file, "coord1", y)
    write(file, "coord2", z)
    write(file, "nvars", [N_fields])
    write(file, "var0", real.(ϕ))
    write(file, "var1", imag.(ϕ))
    write(file, "var2", real.(ϕ_t))
    write(file, "var3", imag.(ϕ_t))
end